In [0]:
from natsort import natsorted
import os
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import load_model,Model,model_from_json
import numpy as np
import cv2
def create_data():
	if not os.path.exists(os.path.join('test')):
		os.mkdir('test')
	cap= cv2.VideoCapture('test.avi')
	i=0
	backsub = cv2.bgsegm.createBackgroundSubtractorMOG()
	bkgrd = cv2.imread('bkgrd.jpg')
	count = 0
	while(cap.isOpened()):
			ret, frame = cap.read()
			if ret == False:
					break
			if i>24 and i%2==0:    
					count+=1
					frame = cv2.absdiff(bkgrd,frame)
					ret,frame = cv2.threshold(frame,30,255,cv2.THRESH_BINARY)
					frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
					frame = backsub.apply(frame)
					cv2.imwrite('./test/frame'+str(count)+'.jpg',frame)
					if count>=40:
							break
			i+=1
	cap.release()
create_data() 
inceptionmodel = load_model('/content/drive/My Drive/MODEL_INCEPTION/inceptionFinal.h5py')
inceptionmodel = Model(inceptionmodel.inputs,inceptionmodel.layers[-3].output)
path = os.path.join('test')
def extract(img_path):
	img = image.load_img(img_path,target_size=(299,299))
	x = image.img_to_array(img)
	del(img)
	x = np.expand_dims(x,axis=0)
	x = preprocess_input(x)
	features = inceptionmodel.predict(x)
	del(x)
	features = features[0]
	return features
a = np.zeros((1,2048))
for f in enumerate(natsorted(os.listdir(path))):
	ftrs = extract(os.path.join(path,f[1]))
	a = np.vstack((a,ftrs))
a = np.delete(a,0,0)
a = np.expand_dims(a,axis=0)
with open('/content/drive/My Drive/LSTM_MODEL/modelLSTM2.json', 'r') as json_file:
	loaded_model_json = json_file.read()
lstm_model = model_from_json(loaded_model_json)
del(loaded_model_json)
lstm_model.load_weights('/content/drive/My Drive/LSTM_MODEL/modelLSTM2_weights.h5')
probabs = lstm_model.predict(a,batch_size=40)
probabs = probabs[0]
top_values_index = sorted(range(len(probabs)), key=lambda i: probabs[i])[-5:]
print(sorted(top_values_index))

[6, 26, 59, 102, 105]
